In [1]:
from tensorflow.contrib.data import Dataset, Iterator
import pandas as pd
import numpy as np
import os
from daewoo_module import *
from tqdm import tqdm_notebook as tqdm

root_dir = ".\\input_data"
# img_dir = os.path.join(root_dir, '/figure'). 이미지는 os.path.join쓰면 안되는경우가 있음
img_dir = "./input_data/figure/"
logs_path = os.path.join(root_dir, "graph")

label = pd.read_csv(os.path.join(root_dir, 'description.csv'), engine='python')
label = label['WVHT ft.y'].values
label = ((label - np.mean(label))/np.std(label)).reshape(-1,1)

batch_size = 16

epochs = 2

C:\Users\Jay\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
img = np.array([img_dir + x for x in os.listdir(img_dir)])

np.random.seed(1234)
idx = np.random.permutation(len(img))
tr_idx = idx[:round(0.8*len(idx))]
ts_idx = idx[round(0.8*len(idx)):]

train_img = img[tr_idx]
train_label = label[tr_idx]
test_img = img[ts_idx]
test_label = label[ts_idx]

train_img_tensor = tf.constant(train_img)
train_label_tensor = tf.constant(train_label)
test_img_tensor = tf.constant(test_img)
test_label_tensor = tf.constant(test_label)

train_imgs = Dataset.from_tensor_slices((train_img_tensor,train_label_tensor))
train_imgs = train_imgs.map(input_tensor).batch(batch_size).shuffle(buffer_size=1000).repeat()
test_imgs = Dataset.from_tensor_slices((test_img_tensor,test_label_tensor))
test_imgs = test_imgs.map(input_tensor).batch(test_batch_size).shuffle(buffer_size=1000).repeat()

train_iterator = train_imgs.make_initializable_iterator()
test_iterator = test_imgs.make_initializable_iterator()
handle = tf.placeholder(tf.string, shape=[])

iterator = tf.data.Iterator.from_string_handle(handle, train_imgs.output_types, train_imgs.output_shapes)
x, y = iterator.get_next()

train_batches = len(train_img) // batch_size
test_batches = len(test_img) // batch_size

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


In [4]:
model_name = "VGG16_"
model = VGG16(x, y, bn=True)

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_handle = sess.run(train_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    train_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'train'), sess.graph)
    test_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'test'))
    
    print("Training!")
    for i in tqdm(range(epochs)):
        print("-------{} Epoch--------".format(i+1))
        sess.run(train_iterator.initializer)
        sess.run(test_iterator.initializer)
        for j in tqdm(range(train_batches)):
            summary, _, loss_ = sess.run([model.merged_summary_op, model.train, model.loss],
                                        feed_dict={handle:train_handle,
                                                   model.learning_rate:0.001})
            if j % 10 == 0:
                print("Training Epoch : {}, Iter : {}, Loss : {:.4f}".format(i+1, j, loss_))
                train_writer.add_summary(summary, j)
                summary, loss_ = sess.run([model.merged_summary_op, model.loss],
                                         feed_dict={handle:test_handle})
                print("Validation Loss : {:.4f}".format(loss_))
                test_writer.add_summary(summary, j)
    
    print("-----------End of training-------------")
            
            

Training!


A Jupyter Widget

-------0 Epoch--------


A Jupyter Widget

Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\Jay\Anaconda3\envs\tensorflow\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Jay\Anaconda3\envs\tensorflow\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\Jay\Anaconda3\envs\tensorflow\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



...

Training Epoch : 1, Iter : 1150, Loss : 1.0975
Validation Loss : 0.7752
Training Epoch : 1, Iter : 1160, Loss : 0.3755
Validation Loss : 0.3501
Training Epoch : 1, Iter : 1170, Loss : 0.9459
Validation Loss : 0.6707
Training Epoch : 1, Iter : 1180, Loss : 1.3763
Validation Loss : 0.7593
Training Epoch : 1, Iter : 1190, Loss : 3.2873
Validation Loss : 0.4565
Training Epoch : 1, Iter : 1200, Loss : 0.7145
Validation Loss : 0.2825
Training Epoch : 1, Iter : 1210, Loss : 0.3517
Validation Loss : 0.3337
Training Epoch : 1, Iter : 1220, Loss : 0.6717
Validation Loss : 0.6977
Training Epoch : 1, Iter : 1230, Loss : 1.9631
Validation Loss : 0.6435
Training Epoch : 1, Iter : 1240, Loss : 0.3831
Validation Loss : 0.6448
Training Epoch : 1, Iter : 1250, Loss : 0.3948
Validation Loss : 0.5341
Training Epoch : 1, Iter : 1260, Loss : 0.9731
Validation Loss : 0.4834
Training Epoch : 1, Iter : 1270, Loss : 0.6006
Validation Loss : 0.2959
Training Epoch : 1, Iter : 1280, Loss : 0.5501
Validation Loss :

Validation Loss : 0.2711
Training Epoch : 1, Iter : 2290, Loss : 1.0395
Validation Loss : 0.7812
Training Epoch : 1, Iter : 2300, Loss : 0.7499
Validation Loss : 0.8652
Training Epoch : 1, Iter : 2310, Loss : 0.9139
Validation Loss : 0.2396
Training Epoch : 1, Iter : 2320, Loss : 0.4904
Validation Loss : 1.2177
Training Epoch : 1, Iter : 2330, Loss : 2.0770
Validation Loss : 0.2561
Training Epoch : 1, Iter : 2340, Loss : 0.9183
Validation Loss : 5.7185
Training Epoch : 1, Iter : 2350, Loss : 0.8713
Validation Loss : 1.6350
Training Epoch : 1, Iter : 2360, Loss : 0.2316
Validation Loss : 3.8807
Training Epoch : 1, Iter : 2370, Loss : 1.0552
Validation Loss : 0.1953
Training Epoch : 1, Iter : 2380, Loss : 2.1350
Validation Loss : 2.2969
Training Epoch : 1, Iter : 2390, Loss : 0.7088
Validation Loss : 0.8308
Training Epoch : 1, Iter : 2400, Loss : 1.2393
Validation Loss : 0.7297
Training Epoch : 1, Iter : 2410, Loss : 0.3550
Validation Loss : 0.8101
Training Epoch : 1, Iter : 2420, Loss :

A Jupyter Widget

Training Epoch : 2, Iter : 0, Loss : 0.9704
Validation Loss : 0.4603
Training Epoch : 2, Iter : 10, Loss : 1.1968
Validation Loss : 0.7484
Training Epoch : 2, Iter : 20, Loss : 2.0195
Validation Loss : 3.3925
Training Epoch : 2, Iter : 30, Loss : 0.5976
Validation Loss : 1.1996
Training Epoch : 2, Iter : 40, Loss : 1.0793
Validation Loss : 1.0127
Training Epoch : 2, Iter : 50, Loss : 0.4296
Validation Loss : 0.7324
Training Epoch : 2, Iter : 60, Loss : 1.5392
Validation Loss : 0.3311
Training Epoch : 2, Iter : 70, Loss : 0.5259
Validation Loss : 0.6492
Training Epoch : 2, Iter : 80, Loss : 0.5868
Validation Loss : 0.1671
Training Epoch : 2, Iter : 90, Loss : 0.3424
Validation Loss : 1.1244
Training Epoch : 2, Iter : 100, Loss : 0.6933
Validation Loss : 1.5143
Training Epoch : 2, Iter : 110, Loss : 0.7186
Validation Loss : 1.1318
Training Epoch : 2, Iter : 120, Loss : 1.2299
Validation Loss : 4.3911
Training Epoch : 2, Iter : 130, Loss : 0.5552
Validation Loss : 0.3104
Training Epoch : 

KeyboardInterrupt: 

In [ ]:
tqdm.monitor_interval = 0

In [ ]:
tf.reset_default_graph()
sess.close()
